In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%load_ext cython

In [ ]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time
import glob

ROOT.gSystem.Load("$MARSSYS/libmars.so")

In [ ]:
path_in_sorcerer  = "/remote/ceph/user/m/mhuetten/MAGIC_data/sorcerer/CrabNebula"
path_in_star  = "/remote/ceph/user/m/mhuetten/MAGIC_data/star/CrabNebula"

filepath_out = "/remote/ceph/user/m/mhuetten/MAGIC_data/ctapipe/"

# Observation data files
filepaths_M1_sorcerer = sorted(glob.glob(path_in_sorcerer + "/*_Y_CrabNebula*.root"))
filepaths_M2_sorcerer = sorted(glob.glob(path_in_sorcerer + "/*_Y_CrabNebula*.root"))

filepaths_M1_star = sorted(glob.glob(path_in_star + "/*_I_CrabNebula*.root"))
filepaths_M2_star = sorted(glob.glob(path_in_star + "/*_I_CrabNebula*.root"))

# Monte Carlo files
#filename_M1 = "GA_M1_za05to35_9_188880_Y_w0.root"
#filename_M2 = "GA_M2_za05to35_9_188880_Y_w0.root"

ntels = 2

nfiles = len(filepaths_M1_sorcerer)

camgeom = ROOT.MGeomCamMagicTwo()
#print camgeom.GetPixels().At(4).Print() 

camera = ROOT.MHCamera(camgeom)

filepaths = [filepaths_M1_sorcerer, filepaths_M2_sorcerer]
filepaths_star = [filepaths_M1_star, filepaths_M2_star]
filename_out = ''.join(''.join(filepaths_M1_sorcerer[0].split("_M1")).split(".001"))
filename_out = os.path.join(filepath_out,filename_out.split("/")[-1])
filename_out = os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5")

In [ ]:
#ROOT.TBrowser()

In [ ]:
def get_ROOT_array(file, tree, branch, leaf):
    tfile = ROOT.TFile(file)
    ttree = getattr(tfile, tree)
    hist = ROOT.TH1D("hist","hist",100,-100,100)
    ttree.Project("hist", "%s.%s"%(branch,leaf),"","",100000000000,0)
    treeArr = ttree.GetV1()
    result = np.array(np.frombuffer(treeArr,count=int(hist.GetEntries())), dtype = np.double)
    return result

In [ ]:
arr_phe_3D = []
arr_arrival_3D = []
StereoEvtNumber_2D = []
TrigPattern_2D = []
DAQEvtNumber_2D = []
ClockCounter_2D = []
TimeDiff_2D = []
MJD_2D = []
Sec_since_MJD_2D = []
Pointings_2D = []
Pointings_corr_2D = []
BadPixels_3D = []

M1_pos = np.array([35., -24., 0])
M2_pos = np.array([-35., 24., 0])
TelPos_2D = [(35, -35), (-24, 24), (0, 0)]

TrigPatternNames = []
# valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
TrigPatternNames.append('L3 trigger')
TrigPatternNames.append('Calibration')
TrigPatternNames.append('LIDAR')
TrigPatternNames.append('Pedestal')
TrigPatternNames.append('Pulse injection')
TrigPatternNames.append('Sumtrigger')
TrigPatternNames.append('Topo trigger')
TrigPatternNames.append('L1 trigger')

n_badpixtype = 26


nevents = np.zeros(2)

for ntel in range(ntels):

    arr_phe_2D = []
    arr_arrival_2D = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []
    BadPixels = []
    
    MJD = []
    Sec_since_MJD = []
    
    Pointings = []
    Pointings_corr = []

    # MC infos
    Energy = [] # TeV
    Core_params = [] # m
    X_max = [] # m
    H_first_int = [] # m
    Shower_angles = []

    

    for nfile in range(2):

        Runheader = ROOT.TChain("RunHeaders")
        Runheader.Add(filepaths[ntel][nfile])
        header_mraw = ROOT.MRawRunHeader()
        header_badpixels = ROOT.MBadPixelsCam()
        Runheader.SetBranchAddress("MRawRunHeader.",header_mraw)
        Runheader.SetBranchAddress("MBadPixelsCam.",header_badpixels)

        Runheader_star = ROOT.TChain("RunHeaders")
        Runheader_star.Add(filepaths_star[ntel][nfile])
        head_observatory = ROOT.MObservatory()
        Runheader_star.SetBranchAddress("MObservatory.",head_observatory)
       
        # extract global information out of first event:
        Runheader.GetEntry(0)
        Runheader_star.GetEntry(0)
        
        TelescopeNumber =  header_mraw.GetTelescopeNumber()

        nevents_tmp = header_mraw.GetNumEvents()
        #nevents_tmp = 100
        #max_event = Events.GetEntries()

        RunNumber = header_mraw.GetRunNumber()
        RunType = header_mraw.GetRunTypeStr()
        if RunType == "Data":
            is_mc = False
        elif RunType == "Monte Carlo":
            is_mc = True
        else:
            is_mc = False
       
        Events = ROOT.TChain("Events")
        Events.Add(filepaths[ntel][nfile])

        evtdata_phe = ROOT.MCerPhotEvt()
        evtdata_time = ROOT.MTime()
        evtdata_arrival = ROOT.MArrivalTime()

        evtheader = ROOT.MRawEvtHeader()
        if is_mc == True:
            mcevtheader = ROOT.MMcEvt()
            
        Events.SetBranchAddress("MCerPhotEvt.",evtdata_phe)
        Events.SetBranchAddress("MArrivalTime.",evtdata_arrival)
        Events.SetBranchAddress("MRawEvtHeader.",evtheader)
        Events.SetBranchAddress("MTime.",evtdata_time)
        if is_mc == True:
            Events.SetBranchAddress("MMcEvt.",mcevtheader)

        if is_mc == False:
            Events_star = ROOT.TChain("Events")
            Events_star.Add(filepaths_star[ntel][nfile])
            evtdata_pointings = ROOT.MPointingPos()
            Events_star.SetBranchAddress("MPointingPos.",evtdata_pointings)
            
        start_time = time.time()

        for nevent in range(nevents_tmp):

            if nevent >= nevents_tmp:
                continue

            sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                             + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(nevents_tmp)))

            Runheader.GetEvent(nevent)
            Events.GetEvent(nevent)
            if is_mc == False:
                Events_star.GetEvent(nevent)

            npixels = evtdata_phe.GetNumPixels()

            trigid = evtheader.GetTriggerID()

            binarypattern = '\t{0:016b}'.format(trigid&0xff) 

            triggpattern = (bool(int(binarypattern[-8])), 
                            bool(int(binarypattern[-1])), 
                            bool(int(binarypattern[-2])), 
                            bool(int(binarypattern[-3])), 
                            bool(int(binarypattern[-4])), 
                            bool(int(binarypattern[-5])), 
                            bool(int(binarypattern[-6])), 
                            bool(int(binarypattern[-7])))

            # check mono-character of run:
            #if nevent > 0:
             #   if triggpattern[0] == False:
             #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
             #          StereoEvtNumber[-1] = -1
                    #else:
                       # print "calib!"

            camera.SetCamContent(evtdata_phe)
            camera.SetAllUsed()
            arr_phe_evt = []
            for i_pixel in range(npixels):
                arr_phe_evt.append(camera.GetBinContent(i_pixel + 1))

            camera.SetCamContent(evtdata_arrival)
            camera.SetAllUsed()
            arr_arrival_evt = []
            for i_pixel in range(npixels):
                arr_arrival_evt.append(camera.GetBinContent(i_pixel + 1))

                
            badpixelvals = np.zeros(shape=(n_badpixtype, npixels), dtype=np.bool)
            for ipixel in range(npixels):
                for i_badpixtype in range(n_badpixtype):
                    val_tmp = ROOT.Double()
                    header_badpixels.GetPixelContent(val_tmp, ipixel, camgeom, i_badpixtype)
                    badpixelvals[i_badpixtype, ipixel] = val_tmp
                
            if arr_phe_evt != []:
                
                nevents[ntel] += 1
                
                arr_phe_2D.append(np.array(arr_phe_evt))
                arr_arrival_2D.append(np.array(arr_arrival_evt))

                DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
                ClockCounter.append(evtheader.GetClockCounter())
                NumTrigLvl2.append(evtheader.GetNumTrigLvl2())
                if is_mc == True:
                    nr = evtheader.GetStereoEvtNumber() 
                    if nr != 0:
                        StereoEvtNumber.append(nr + nfile * 1000)
                    else:
                        StereoEvtNumber.append(nr)
                else:
                    StereoEvtNumber.append(evtheader.GetStereoEvtNumber())
                TriggerID.append(evtheader.GetTriggerID())
                TrigPattern.append(triggpattern)
                CalibrationPattern.append(evtheader.GetCalibrationPattern())
                MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
                TimeDiff.append(evtheader.GetTimeDiff())
                MJD.append(np.floor(evtdata_time.GetMjd()))
                Sec_since_MJD.append(evtdata_time.GetTime()/1e3 + evtdata_time.GetNanoSec()/1e9)
                # so far, there does not seem to be difference between the bad pixels event by event
                if BadPixels == []:
                    BadPixels.append(badpixelvals)
                
                if is_mc == False:
                    Pointings.append([evtdata_pointings.GetAz(), evtdata_pointings.GetZd()])
                    Pointings_corr.append([evtdata_pointings.GetCorrAz(), evtdata_pointings.GetCorrZd()])
                else:
                    Pointings.append([np.rad2deg(mcevtheader.GetTelescopePhi()), 
                                      90. - np.rad2deg(mcevtheader.GetTelescopeTheta())])
                    Pointings_corr.append([np.nan, np.nan])
                    
                    energy_val = str(mcevtheader.GetEnergyStr())[:-3]
                    energy_unit = str(mcevtheader.GetEnergyStr())[-3:]
                    if energy_unit == "TeV":
                        Energy.append(float(energy_val))
                    elif energy_unit == "GeV":
                        Energy.append(float(energy_val) / 1000.)
                    else:
                        raise IOError("Error!")
                    Core_params.append((mcevtheader.GetCoreX(), mcevtheader.GetCoreX())) # meters
                    H_first_int.append(mcevtheader.GetZFirstInteraction()) # meters

        end_time = time.time()
        elapsed = end_time - start_time
        print "\n File %s read into python arrays >>> Elapsed time: %.1f s" %(filepaths[ntel][nfile], elapsed)
                    
    
    # give MC mono events an ordered event number:
    if is_mc == True:
        dt_events = np.double
        for i_event in range(len(StereoEvtNumber)):
            if StereoEvtNumber[i_event] == 0:
                if i_event != 0:
                    StereoEvtNumber[i_event] = np.random.uniform(StereoEvtNumber[i_event - 1], 
                                                                 np.floor(StereoEvtNumber[i_event - 1]) + 1)
                else:
                    StereoEvtNumber[i_event] = np.random.uniform(0, 1)
    else:
        dt_events = np.int32
        
    StereoEvtNumber_2D.append(np.array(StereoEvtNumber, dtype=dt_events))
    DAQEvtNumber_2D.append(np.array(DAQEvtNumber, dtype=np.int32))
    arr_phe_3D.append(np.array(arr_phe_2D))
    arr_arrival_3D.append(np.array(arr_arrival_2D))
    TrigPattern_2D.append(TrigPattern)
    MJD_2D.append(MJD)
    Sec_since_MJD_2D.append(Sec_since_MJD)
    ClockCounter_2D.append(np.array(ClockCounter, dtype=np.int64))
    TimeDiff_2D.append(np.array(TimeDiff))
    Pointings_2D.append(Pointings)
    Pointings_corr_2D.append(Pointings_corr)
    BadPixels_3D.append(np.array(BadPixels, dtype=np.bool))

In [ ]:
print nevents[1]

In [ ]:
# Separate data and pedestal events and make stereo events match:

arr_phe_3D_data = []
arr_arrival_3D_data = []
#arr_badpixels_4D_data = []
tels_with_data = []
gps_time = []
pointing = []
eventstream_corrected = [] # some stereo events are double pedestal triggers

# all data events:
eventstream = np.union1d(StereoEvtNumber_2D[0], StereoEvtNumber_2D[1])

# all stereo events:
stereoevents = np.intersect1d(StereoEvtNumber_2D[0], StereoEvtNumber_2D[1])
nstereo = len(stereoevents)
nstereo_check = 0

arr_badpixels_3D_data = np.zeros((2,n_badpixtype,npixels,), dtype = np.bool)
arr_badpixels_3D_data[0] = BadPixels_3D[0][0]
arr_badpixels_3D_data[1] = BadPixels_3D[1][0]

for event_id in eventstream:

    sys.stdout.write("\r ... processing data event " + str(event_id))

    stereopatterns = np.zeros(2, dtype=np.bool)
    stereotimestamps = np.zeros(shape=(2,2)) - 1.
        
    arr_phe_2D_data = np.empty((2,npixels,))
    arr_phe_2D_data[:] = None
    arr_arrival_2D_data = np.empty((2,npixels,))
    arr_arrival_2D_data[:] = None
    
    
    pointings = np.empty((2,2,2))
    pointings[:] = None
    
    for ntel in range(ntels):
        
        # search event
        if event_id not in StereoEvtNumber_2D[ntel]:
            nevent = -1
        else:
            nevent = np.searchsorted(StereoEvtNumber_2D[ntel], event_id, side='left')
            
        # sort out remaining calibration runs:
        if TrigPattern_2D[ntel][nevent][0] == False:
            nevent = -1    
    
        if nevent != -1:
            arr_phe_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
            arr_arrival_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
            stereopatterns[ntel] = True
            stereotimestamps[ntel][0] = MJD_2D[ntel][nevent]
            stereotimestamps[ntel][1] = Sec_since_MJD_2D[ntel][nevent]
  
            pointings[ntel,0,0] = Pointings_2D[ntel][nevent][0]
            pointings[ntel,1,0] = Pointings_2D[ntel][nevent][1]
            pointings[ntel,0,1] = Pointings_corr_2D[ntel][nevent][0]
            pointings[ntel,1,1] = Pointings_corr_2D[ntel][nevent][1]

    if (stereopatterns == [False, False]).all():
        # no stereo event, accidental double pedestal trigger
        continue
    elif (stereopatterns == [False, True]).all():
        tels_with_data.append((False, True))
    elif (stereopatterns == [True, False]).all():
        tels_with_data.append((True, False))
    elif (stereopatterns == [True, True]).all():
        tels_with_data.append((True, True))   
        
    gps_time.append((stereotimestamps[0][0], stereotimestamps[0][1], stereotimestamps[1][0], stereotimestamps[1][1]))
    arr_phe_3D_data.append(arr_phe_2D_data)
    arr_arrival_3D_data.append(arr_arrival_2D_data)
    
    # so far, there does not seem to be difference between the bad pixels event by event 
    #arr_badpixels_4D_data.append(arr_badpixels_3D_data)
    
    pointing.append((pointings[0,0,0], pointings[0,0,1], pointings[0,1,0], pointings[0,1,1], 
                     pointings[1,0,0], pointings[1,0,1], pointings[1,1,0], pointings[1,1,1]))
    eventstream_corrected.append(event_id)


nevents_data = len(eventstream_corrected)


if is_mc == False:
    # loop again through all events to catch the pedestal events:
    event_id_pedestal = 0
    for ntel in range(ntels):
        for nevent in range(len(StereoEvtNumber_2D[ntel])):

            if TrigPattern_2D[ntel][nevent][3] == True:

                event_id_pedestal -= 1 # count pedestal events to negative

                sys.stdout.write("\r ... processing pedestal event " + str(event_id_pedestal))

                if ntel == 0:
                    tels_with_data.append((True, False))
                else:
                    tels_with_data.append((False, True))

                stereotimestamps = np.zeros(shape=(2,2)) - 1.
                stereotimestamps[ntel][0] = MJD_2D[ntel][nevent]
                stereotimestamps[ntel][1] = Sec_since_MJD_2D[ntel][nevent]

                pointings = np.empty((2,2,2))
                pointings[:] = None

                pointings[ntel,0,0] = Pointings_2D[ntel][nevent][0]
                pointings[ntel,1,0] = Pointings_2D[ntel][nevent][1]
                pointings[ntel,0,1] = Pointings_corr_2D[ntel][nevent][0]
                pointings[ntel,1,1] = Pointings_corr_2D[ntel][nevent][1]

                #arr_badpixels_3D_data[ntel] = BadPixels_3D[ntel][nevent]

                gps_time.append((stereotimestamps[0][0], stereotimestamps[0][1], stereotimestamps[1][0], stereotimestamps[1][1]))
                eventstream_corrected.append(event_id_pedestal)

                arr_phe_2D_data = np.empty((2,npixels,))
                arr_phe_2D_data[:] = None
                arr_arrival_2D_data = np.empty((2,npixels,))
                arr_arrival_2D_data[:] = None

                arr_phe_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
                arr_arrival_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
                arr_phe_3D_data.append(arr_phe_2D_data)
                arr_arrival_3D_data.append(arr_arrival_2D_data)   
                #arr_badpixels_4D_data.append(arr_badpixels_3D_data)

                pointing.append((pointings[0,0,0], pointings[0,0,1], pointings[0,1,0], pointings[0,1,1], 
                                 pointings[1,0,0], pointings[1,0,1], pointings[1,1,0], pointings[1,1,1]))

arr_phe_3D_data = np.array(arr_phe_3D_data)
arr_arrival_3D_data = np.array(arr_arrival_3D_data)

eventstream_corrected = np.array(eventstream_corrected, dtype=dt_events)
nevents_total = len(eventstream_corrected)           

In [ ]:
nevents_total

In [ ]:
f = h5py.File(filename_out,"w")

f.attrs['Nevents_data'] = nevents_data
f.attrs['Nevents_pedestal'] = nevents_total - nevents_data
f.attrs['Nevents_total'] = nevents_total
f.attrs['dl_export'] = np.string_("dl1")
f.attrs['dl_export_comment'] = np.string_("MAGIC Sorcerer output")
f.attrs['data format'] = np.string_("stereo")
#f.attrs['data format'] = np.string_("mono M1")
#f.attrs['data format'] = np.string_("mono M2")
f.attrs['instrument'] = np.string_("MAGIC")
f.attrs['RunType'] = RunType
f.attrs['RunNumber'] = RunNumber

dsets = []
dsets.append(f.create_dataset("dl1/event_id", data=eventstream_corrected))#, compression_opts=9)

dt = np.dtype([("M1", np.bool), ("M2", np.bool)])
dsets.append(f.create_dataset("trig/tels_with_trigger", (nevents_total,), dt))
dsets[-1][...] = np.array(tels_with_data, dtype = dt)
f["/dl1/tels_with_data"] = h5py.SoftLink('/trig/tels_with_trigger')

dt = np.dtype([("M1_Az", np.double), ("M1_AzCorr", np.double), ("M1_Dec", np.double), ("M1_DecCorr", np.double), 
               ("M2_Az", np.double), ("M2_AzCorr", np.double), ("M2_Dec", np.double), ("M2_DecCorr", np.double)])
dsets.append(f.create_dataset("pointing", (nevents_total,), dt))
dsets[-1][...] = np.array(pointing, dtype = dt)
dsets[-1].attrs['FIELD_0_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_1_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_2_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_3_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_4_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_5_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_6_UNIT'] = np.string_("deg")
dsets[-1].attrs['FIELD_7_UNIT'] = np.string_("deg")

dt = np.dtype([("M1_mjd", np.int32), ("M1_sec", np.double), ("M2_mjd", np.int32), ("M2_sec", np.double)])
dsets.append(f.create_dataset("trig/gps_time", (nevents_total,), dt))
dsets[-1][...] = np.array(gps_time, dtype = dt)
dsets[-1].attrs['FIELD_0_UNIT'] = np.string_("d")
dsets[-1].attrs['FIELD_1_UNIT'] = np.string_("s")
dsets[-1].attrs['FIELD_2_UNIT'] = np.string_("d")
dsets[-1].attrs['FIELD_3_UNIT'] = np.string_("s")

dt = np.dtype([("M1", np.double), ("M2", np.double)])
dsets.append(f.create_dataset("inst/subarray/tel_coords", (3,), dt))
dsets[-1][...] = np.array(TelPos_2D, dtype = dt)
dsets[-1].attrs['FIELD_0_UNIT'] = np.string_("m")
dsets[-1].attrs['FIELD_1_UNIT'] = np.string_("m")
dsets[-1].attrs['FIELD_0_NAME'] = np.string_("coordinates (x,y,z)")
dsets[-1].attrs['FIELD_1_NAME'] = np.string_("coordinates (x,y,z)")
f["inst/subarray"].attrs['OpticsDescription'] = np.string_("MAGIC")
f["inst/subarray"].attrs['CameraGeometry'] = np.string_("MAGICCam")

for ntel in range(ntels):
    
    ##########################
    # write data to hdf5 file:
    

    
    dsets.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/image", data=arr_phe_3D_data[:,ntel,:], compression='gzip'))#, compression_opts=9)
    dsets.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/peakpos", data=arr_arrival_3D_data[:,ntel,:], compression='gzip'))#, compression_opts=9)
    dsets.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/badpixels", data=arr_badpixels_3D_data[ntel].transpose(), compression='gzip'))#, compression_opts=9)
    dsets[-1].attrs['FIELD_00_NAME'] = np.string_("UnsuitableRun")
    dsets[-1].attrs['FIELD_01_NAME'] = np.string_("UnsuitableEvt")
    dsets[-1].attrs['FIELD_02_NAME'] = np.string_("UnreliableRun")
    dsets[-1].attrs['FIELD_03_NAME'] = np.string_("HiGainBad")
    dsets[-1].attrs['FIELD_04_NAME'] = np.string_("LoGainBad")
    dsets[-1].attrs['FIELD_05_NAME'] = np.string_("UnsuitableCalLevel")
    dsets[-1].attrs['FIELD_06_NAME'] = np.string_("UnreliableCalLevel")
    dsets[-1].attrs['FIELD_07_NAME'] = np.string_("Uncalibrated:HiGainNotFitted")
    dsets[-1].attrs['FIELD_08_NAME'] = np.string_("Uncalibrated:HiLoGainNotFitted")
    dsets[-1].attrs['FIELD_09_NAME'] = np.string_("Uncalibrated:HiGainOscillating")
    dsets[-1].attrs['FIELD_10_NAME'] = np.string_("Uncalibrated:LoGainOscillating")
    dsets[-1].attrs['FIELD_11_NAME'] = np.string_("Uncalibrated:LoGainSaturation")
    dsets[-1].attrs['FIELD_12_NAME'] = np.string_("Uncalibrated:ChargeIsPedestal")
    dsets[-1].attrs['FIELD_13_NAME'] = np.string_("Uncalibrated:ChargeErrNotValid")
    dsets[-1].attrs['FIELD_14_NAME'] = np.string_("Uncalibrated:ChargeRelErrNotValid")
    dsets[-1].attrs['FIELD_15_NAME'] = np.string_("Uncalibrated:ChargeSigmaNotValid")
    dsets[-1].attrs['FIELD_16_NAME'] = np.string_("Uncalibrated:MeanTimeInFirstBin")
    dsets[-1].attrs['FIELD_17_NAME'] = np.string_("Uncalibrated:MeanTimeInLast2Bins")
    dsets[-1].attrs['FIELD_18_NAME'] = np.string_("Uncalibrated:DeviatingNumPhes")
    dsets[-1].attrs['FIELD_19_NAME'] = np.string_("Uncalibrated:RelTimeNotFitted")
    dsets[-1].attrs['FIELD_20_NAME'] = np.string_("Uncalibrated:RelTimeOscillating")
    dsets[-1].attrs['FIELD_21_NAME'] = np.string_("Uncalibrated:DeviatingNumPhots")
    dsets[-1].attrs['FIELD_22_NAME'] = np.string_("Uncalibrated:HiGainOverFlow")
    dsets[-1].attrs['FIELD_23_NAME'] = np.string_("Uncalibrated:LoGainOverFlow")
    dsets[-1].attrs['FIELD_24_NAME'] = np.string_("Uncalibrated:UnsuitableDC")
    dsets[-1].attrs['FIELD_25_NAME'] = np.string_("Unsuitable")
    
    # Raw-level data with non-data events for cross checks:
    dt = np.dtype([(TrigPatternNames[0], np.bool), (TrigPatternNames[1], np.bool), (TrigPatternNames[2], np.bool), 
                   (TrigPatternNames[3], np.bool), (TrigPatternNames[4], np.bool), (TrigPatternNames[5], np.bool), 
                   (TrigPatternNames[6], np.bool), (TrigPatternNames[7], np.bool)])
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/TrigPattern", (nevents[ntel],), dt))
    dsets[-1][...] = np.array(TrigPattern_2D[ntel], dtype = dt)
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/DAQEvtNumber", data=DAQEvtNumber_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/StereoEvtNumber", data=StereoEvtNumber_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/ClockCounter", data=ClockCounter_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/TimeDiff", data=TimeDiff_2D[ntel]))
    
    # write MC information:
    if is_mc == True and ntel == 1:
        dsets_mcheader = []
        
        dt = np.dtype([("Energy", np.double),])
        dsets_mcheader.append(f.create_dataset("McHeader/Energy", (nevents[ntel],), dt))
        dsets_mcheader[-1][...] = np.array(Energy, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = np.string_("Energy")
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = np.string_("TeV")
        
        dt = np.dtype([("Core_x", np.double), ("Core_y", np.double)])
        dsets_mcheader.append(f.create_dataset("McHeader/Core_xy", (nevents[ntel],), dt))
        dsets_mcheader[-1][...] = np.array(Core_params, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = np.string_("Core_x")
        dsets_mcheader[-1].attrs['FIELD_1_NAME'] = np.string_("Core_y")
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = np.string_("m")

        dt = np.dtype([("H_first_int", np.double),])
        dsets_mcheader.append(f.create_dataset("McHeader/H_first_int", (nevents[ntel],), dt))
        dsets_mcheader[-1][...] = np.array(H_first_int, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = np.string_("H_first_int")
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = np.string_("m")
        

f.close()

In [ ]:
f.close()